In [2]:
import numpy as np
from digitalio import DigitalInOut, Direction
from adafruit_rgb_display import st7789
import board
from PIL import Image, ImageDraw, ImageFont, ImageOps
import time
import random
import cv2 as cv
import numpy as np
from colorsys import hsv_to_rgb

In [3]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        self.width = self.disp.width
        self.height = self.disp.height

        # 배경 이미지 리스트 생성
        self.bg_images = ['bg/bg1.png', 'bg/bg2.png', 'bg/bg3.png']
        self.bg_images = [Image.open(bg).resize((self.width, self.height)).rotate(180) for bg in self.bg_images]
        self.bg_index = 0  # 현재 배경 이미지 인덱스


In [4]:
level1Fish = [
    "fish/f_1_1.png", "fish/f_1_2.png", "fish/f_1_3.png", "fish/f_1_4.png", "fish/f_1_5.png"]
level2Fish = [
    "fish/f_2_1.png", "fish/f_2_2.png", "fish/f_2_3.png", "fish/f_poison.png"]
level3Fish = [
    "fish/f_3_1.png", "fish/f_3_2.png", "fish/f_3_3.png", "fish/f_poison.png"]
#레벨 2, 3 에서만 독물고기 생성

class Fish:
    def __init__(self, level, width, height):
        self.level = level
        if level == 1:
            self.appearance = Image.open(random.choice(level1Fish))
        elif level == 2:
            self.appearance = Image.open(random.choice(level2Fish))
        elif level == 3:
            self.appearance = Image.open(random.choice(level3Fish))
        self.width = width
        self.height = height
        self.generate_random_fish()


    def generate_random_fish(self):
        fish_sizes = [(25, 25), (35, 35), (45, 45)]  # 레벨에 따른 물고기 크기
        base_image = self.appearance.resize(fish_sizes[self.level - 1])
        base_image = ImageOps.flip(base_image) 
        
        self.appearances = {
            'left': ImageOps.mirror(base_image),
            'right': base_image
        }
        self.appearance = self.appearances[random.choice(['left', 'right'])]
        x = random.randint(0, self.width - fish_sizes[self.level - 1][0])
        y = random.randint(0, self.height - fish_sizes[self.level - 1][1])
        self.position = np.array([x, y, x + fish_sizes[self.level - 1][0], y + fish_sizes[self.level - 1][1]])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"

    def move(self):
        if self.appearance == self.appearances['left']:
            self.position[0] -= 3
            self.position[2] -= 3
            if self.position[0] <= 0:
                self.appearance = self.appearances['right']
        else:
            self.position[0] += 3
            self.position[2] += 3
            if self.position[2] >= self.width:
                self.appearance = self.appearances['left']
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

    def get_score(self):
        # 레벨에 따른 점수 반환
        if self.level == 1:
            return 10
        elif self.level == 2:
            return 30
        elif self.level == 3:
            return 50
    

In [5]:
from PIL import Image, ImageOps

class Nemo:
    def __init__(self, width, height, joystick):
        self.level = 1
        self.load_images()
        self.set_level(self.level)  # 이 부분에서 이미지 로딩 및 크기 조절이 이루어집니다.

        self.state = None
        self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 + 20])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"
        self.joystick = joystick

    def load_images(self):
        # 모든 레벨에 대한 이미지를 미리 로드하고 크기를 조절합니다.
        self.all_images = {}
        for level in range(1, 4):
            if level == 1: size = 50
            elif level == 2: size = 65
            else: size = 80

            base_image = Image.open('fish/f_nemo.png').resize((size, size))
            self.all_images[level] = {
                'up': base_image.rotate(270),
                'down': base_image.rotate(90),
                'left': ImageOps.mirror(base_image.rotate(180)),
                'right': base_image.rotate(180)  
            }

    def set_level(self, level):
        self.level = level
        self.appearances = self.all_images[level]  # 레벨에 맞는 이미지를 선택합니다.
        self.appearance = self.appearances['left']

    def move(self, command=None):
        if command['move'] == False:
            self.state = None
            self.outline = "#FFFFFF"  # 검정색상 코드!
        else:
            self.state = 'move'
            self.outline = "#FF0000"  # 빨강색상 코드!

            if command['up_pressed']:
                if (self.position[1] > 0) or (self.position[1] <= 0 and self.joystick.bg_index < len(self.joystick.bg_images) - 1):  # bg1의 맨 밑에 도달하면 아래로 더 내려갈 수 있음
                    self.position[1] -= 5
                    self.position[3] -= 5
                    self.appearance = self.appearances['up']  # 위쪽으로 움직일 때 이미지 변경

            if command['down_pressed']:
                if (self.position[3] < self.joystick.height) or (self.position[3] >= self.joystick.height and self.joystick.bg_index > 0):  # bg3에서는 위로는 bg2로 전환될 수 있지만 배경의 끝이므로 아래는 갈 수 없음
                    self.position[1] += 5
                    self.position[3] += 5
                    self.appearance = self.appearances['down']  # 아래쪽으로 움직일 때 이미지 변경

            if command['left_pressed']:
                if self.position[0] > 0:  # 화면 왼쪽 끝에 도달하지 않았을 때만 이동 가능
                    self.position[0] -= 5
                    self.position[2] -= 5
                    self.appearance = self.appearances['left']  # 왼쪽으로 움직일 때 이미지 변경

            if command['right_pressed']:
                if self.position[2] < self.joystick.width:  # 화면 오른쪽 끝에 도달하지 않았을 때만 이동 가능
                    self.position[0] += 5
                    self.position[2] += 5
                    self.appearance = self.appearances['right']  # 오른쪽으로 움직일 때 이미지 변경
                
        #center update
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2]) 


In [6]:
def draw_game_screen(my_image, my_Nemo, joystick, fish_list, score):
    draw = ImageDraw.Draw(my_image)

    my_image.paste(change_background(my_Nemo, joystick, fish_list))  
    my_image.paste(my_Nemo.appearance, (int(my_Nemo.position[0]), int(my_Nemo.position[1])), my_Nemo.appearance)

    for fish in fish_list:
        my_image.paste(fish.appearance, (int(fish.position[0]), int(fish.position[1])), fish.appearance)

    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"  # 폰트 경로 설정
    font = ImageFont.truetype(font_path, 15)  # 폰트 타입과 크기 설정

    score_text = f"Score: {score}"
    score_width, score_height = draw.textsize(score_text, font=font)
    score_image = Image.new('RGBA', (score_width, score_height), (0, 0, 0, 0))  
    score_draw = ImageDraw.Draw(score_image)
    score_draw.text((0, 0), score_text, font=font, fill="black")
    score_image = score_image.rotate(180)

    my_image.paste(score_image, (170, 220), score_image)
    joystick.disp.image(my_image)


def clear_fish(joystick, fish_list):
    fish_list.clear()  # 기존 물고기 리스트를 비움


def generate_fish(joystick, fish_list):
    if joystick.bg_index == 0:  # bg1에서는 레벨 1의 물고기만 생성
        fish_list.append(Fish(1, joystick.width, joystick.height))
    elif joystick.bg_index == 1:  # bg2에서는 레벨 2의 물고기만 생성
        fish_list.append(Fish(2, joystick.width, joystick.height))
    elif joystick.bg_index == 2:  # bg3에서는 레벨 3의 물고기만 생성
        fish_list.append(Fish(3, joystick.width, joystick.height))


def change_background(my_Nemo, joystick, fish_list):
    if my_Nemo.position[1] <= 0 and joystick.bg_index < len(joystick.bg_images) - 1:  
        joystick.bg_index += 1  
        my_Nemo.position[1] = joystick.height - my_Nemo.appearance.height  
        my_Nemo.position[3] = joystick.height  
        clear_fish(joystick, fish_list)
        generate_fish(joystick, fish_list)
    elif my_Nemo.position[3] >= joystick.height and joystick.bg_index > 0:  
        joystick.bg_index -= 1  
        my_Nemo.position[1] = 0  
        my_Nemo.position[3] = my_Nemo.appearance.height  
        clear_fish(joystick, fish_list)
        generate_fish(joystick, fish_list)

    return joystick.bg_images[joystick.bg_index]


def calculate_distance(point1, point2):
    distance = np.sqrt(np.sum((point1 - point2)**2))
    #print(f"Distance: {distance}, Point1: {point1}, Point2: {point2}")  # 로그 출력
    return distance


def eat_fish(nemo, fish):
    distance = calculate_distance(nemo.center, fish.center)
    return distance < 20 and nemo.level >= fish.level


In [7]:
def main():
    joystick = Joystick()
    my_image = Image.new("RGB", (joystick.width, joystick.height))
    my_draw = ImageDraw.Draw(my_image)
    
    nemoLevel=1
    my_Nemo = Nemo(joystick.width, joystick.height, joystick)
    fish_list = []
    last_fish_time = time.time()
    score=0

    while True:
        command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
        
        if not joystick.button_U.value:  
            command['up_pressed'] = True
            command['move'] = True

        if not joystick.button_D.value:  
            command['down_pressed'] = True
            command['move'] = True

        if not joystick.button_L.value:  
            command['left_pressed'] = True
            command['move'] = True

        if not joystick.button_R.value:  
            command['right_pressed'] = True
            command['move'] = True

        my_Nemo.move(command)

        current_time = time.time()
        if current_time - last_fish_time > 5:  
            generate_fish(joystick, fish_list)  # 배경에 맞는 물고기 생성
            last_fish_time = current_time


        i = 0
        while i < len(fish_list):
            fish = fish_list[i]
            fish.move()
            if eat_fish(my_Nemo, fish):
                score += fish.get_score()
                del fish_list[i]
                if score > 30:
                    my_Nemo.set_level(3)
                elif score > 10:
                    my_Nemo.set_level(2)
            else:
                i += 1

        
        draw_game_screen(my_image, my_Nemo, joystick, fish_list, score)

if __name__ == '__main__':
    main()

KeyboardInterrupt: 